In [1]:
import requests
from twisted.internet import reactor
from multiprocessing import Process, Queue
import scrapy
from scrapy.crawler import CrawlerRunner
from bs4 import BeautifulSoup
from scrapy.utils.project import get_project_settings

settings = get_project_settings()
settings['FEED_FORMAT'] = 'json'
settings['FEED_URI'] = 'result2.json'
settings['RANDOMIZE_DOWNLOAD_DELAY'] = True
settings['DOWNLOAD_DELAY'] = 0

import re

class MySpider(scrapy.Spider):
    name = "Indeed2"
    def __init__(self): 
        self.count = 0
        self.urls = [
          "https://www.indeed.com.ph/Bpo-Hiring-jobs-in-Makati"
        ]

    def start_requests(self):
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        print("processing", response.url)
        soup = BeautifulSoup(response.body, 'html.parser')
        loc = soup.find(id='LOCATION_rbo')
        for le in list(loc.find_all("li")):
            if le.find(class_="rbCount") != None and le.find(class_="rbLabel") != None:
                count = le.find(class_="rbCount").string
                location = le.find(class_="rbLabel").string
                yield {"c": count, "l": location}

        print("yielding next url")
        yield scrapy.Request("https://www.indeed.com.ph/jobs?q=Bpo+Hiring&l=Makati&start=10", callback=self.parse)
        self.count += 1

def run_spider():
    def stop_now():
        print('stopping ', reactor)
        reactor.stop()

    def f(q):
        try:
            runner = CrawlerRunner(settings)
            d = runner.crawl(MySpider)
            d.addBoth(lambda _: stop_now())

            reactor.run()
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

      
run_spider()

processing https://www.indeed.com.ph/Bpo-Hiring-jobs-in-Makati
yielding next url
processing https://www.indeed.com.ph/jobs?q=Bpo+Hiring&l=Makati&start=10
yielding next url
stopping  <twisted.internet.epollreactor.EPollReactor object at 0x7f3e294e4940>


In [2]:
!cat result2.json

[
{"c": "(1747)", "l": "Manila"},
{"c": "(1205)", "l": "Makati"},
{"c": "(815)", "l": "Taguig"},
{"c": "(626)", "l": "Mandaluyong"},
{"c": "(620)", "l": "Pasig"},
{"c": "(544)", "l": "Quezon City"},
{"c": "(481)", "l": "Alabang"},
{"c": "(295)", "l": "Muntinlupa"},
{"c": "(261)", "l": "National Capital Region"},
{"c": "(226)", "l": "Pasay"},
{"c": "(182)", "l": "Ortigas"},
{"c": "(129)", "l": "Fort Bonifacio"},
{"c": "(90)", "l": "Eastwood City"},
{"c": "(74)", "l": "Para\u00f1aque"},
{"c": "(72)", "l": "Cavite City"},
{"c": "nationwide", "l": "Bpo Hiring jobs"},
{"c": "(1747)", "l": "Manila"},
{"c": "(1205)", "l": "Makati"},
{"c": "(815)", "l": "Taguig"},
{"c": "(626)", "l": "Mandaluyong"},
{"c": "(620)", "l": "Pasig"},
{"c": "(544)", "l": "Quezon City"},
{"c": "(481)", "l": "Alabang"},
{"c": "(295)", "l": "Muntinlupa"},
{"c": "(261)", "l": "National Capital Region"},
{"c": "(226)", "l": "Pasay"},
{"c": "(182)", "l": "Ortigas"},
{"c": "(129)", "l": "Fort Bonifacio"},
{"c": "(90)", "l"

In [3]:
import pandas as pd
dfjson = pd.read_json('result2.json')
dfjson

,c,l
0,(1747),Manila
1,(1205),Makati
2,(815),Taguig
3,(626),Mandaluyong
4,(620),Pasig
5,(544),Quezon City
6,(481),Alabang
7,(295),Muntinlupa
8,(261),National Capital Region
9,(226),Pasay


In [ ]:
from twisted.internet import reactor
from multiprocessing import Process, Queue
import scrapy
from scrapy.crawler import CrawlerRunner
from bs4 import BeautifulSoup
from scrapy.utils.project import get_project_settings

settings = get_project_settings()
settings['FEED_FORMAT'] = 'json'
settings['FEED_URI'] = 'tripadvisor.json'
settings['RANDOMIZE_DOWNLOAD_DELAY'] = True
settings['DOWNLOAD_DELAY'] = 0

import re

class MySpider(scrapy.Spider):
    name = "TripAdvisor"
    def __init__(self): 
        self.count = 0
        self.urls = [
          "https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-Church_of_St_George-Lalibela_Amhara_Region.html"
        ]

    def start_requests(self):
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        print("processing", response.url)
        soup = BeautifulSoup(response.body, 'html.parser')
        rev = soup.find(id='ui_column is-9')
        for le in list(rev.find_all("li")):
            if le.find(class_="noQuotes") != None and le.find(class_="partial_entry") != None:
                heads = le.find(class_="noQuotes").string
                bodies = le.find(class_="partial_entry").string
                yield {"h": count, "b": location}
            if 

        next_page = response.css('li.next a::attr("href")').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

def run_spider():
    def stop_now():
        print('stopping ', reactor)
        reactor.stop()

    def f(q):
        try:
            runner = CrawlerRunner(settings)
            d = runner.crawl(MySpider)
            d.addBoth(lambda _: stop_now())

            reactor.run()
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

      
run_spider()

In [11]:
from twisted.internet import reactor
from multiprocessing import Process, Queue
import scrapy
from scrapy.crawler import CrawlerRunner
from bs4 import BeautifulSoup
from scrapy.utils.project import get_project_settings

settings = get_project_settings()
settings['FEED_FORMAT'] = 'json'
settings['FEED_URI'] = 'tripadvisor.json'
settings['RANDOMIZE_DOWNLOAD_DELAY'] = True
settings['DOWNLOAD_DELAY'] = 0

import re

class MySpider(scrapy.Spider):
    name = "TripAdvisor"
    def __init__(self): 
        self.count = 0
        self.urls = [
          "https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-Church_of_St_George-Lalibela_Amhara_Region.html"
        ]

    def start_requests(self):
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        print("processing", response.url)
#         soup = BeautifulSoup(response.body, 'html.parser')
#         rev = soup.find(id='ui_column is-9')
#         for le in list(rev.find_all("li")):
#             if le.find(class_="noQuotes") != None and le.find(class_="partial_entry") != None:
#                 heads = le.find(class_="noQuotes").string
#                 bodies = le.find(class_="partial_entry").string
#                 yield {"h": count, "b": location}
#             if 

#         next_page = response.css('li.next a::attr("href")').get()
#         if next_page is not None:
#             yield response.follow(next_page, self.parse)
        print('1')
        for review in response.css('div.prw_rup prw_reviews_review_resp'):
            print('2')
            yield {
                'head': review.css('span.noQuotes::text').get(),
                'body': review.css('span.partial_entry::text').get(),
                'date': review.xpath('div.prw_rup prw_reviews_stay_date_hsx/text()').get(),
            }

        next_page = response.css('div.unified ui_pagination a::attr("href")').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

def run_spider():
    def stop_now():
        print('stopping ', reactor)
        reactor.stop()

    def f(q):
        try:
            runner = CrawlerRunner(settings)
            d = runner.crawl(MySpider)
            d.addBoth(lambda _: stop_now())

            reactor.run()
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

      
run_spider()

processing https://www.tripadvisor.com.ph/Attraction_Review-g480193-d10147730-Reviews-Church_of_St_George-Lalibela_Amhara_Region.html
1
stopping  <twisted.internet.epollreactor.EPollReactor object at 0x7f3e294e4940>


In [ ]:
<div class="prw_rup prw_reviews_review_resp" data-prwidget-name="reviews_review_resp" data-prwidget-init="handlers"><div id="review_724131885" class="reviewSelector" data-reviewid="724131885"><div class="rev_wrap ui_columns is-multiline"><div class="ui_column is-2 memberInfoColumn"><div class="prw_rup prw_reviews_member_info_resp" data-prwidget-name="reviews_member_info_resp" data-prwidget-init="handlers"><div class="member_info"><div id="UID_BFAD18ACB4FFC44B3055326623451085-SRC_724131885" class="memberOverlayLink clickable" onclick="widgetEvCall('handlers.initMemberOverlay', event, this);" data-anchorwidth="90"><div class="avatar profile_BFAD18ACB4FFC44B3055326623451085"><div class="avatarWrapper"><a><div class="prw_rup prw_common_basic_image avatarImage" data-prwidget-name="common_basic_image" data-prwidget-init=""><div class="ui_avatar resp"> <img src="https://media-cdn.tripadvisor.com/media/photo-l/04/78/16/82/harry-h.jpg" class="basicImg" data-mediaid="74978946"></div></div></a></div></div><div class="info_text pointer_cursor" onclick="widgetEvCall('handlers.usernameClick', event, this);"><div>Harry H</div><div class="userLoc"><strong>Evesham, England, United Kingdom</strong></div></div></div><div id="UID_BFAD18ACB4FFC44B3055326623451085-SRC_724131885" class="memberOverlayLink clickable" onclick="widgetEvCall('handlers.initMemberOverlay', event, this);" data-anchorwidth="90"><div class="memberBadgingNoText is-shown-at-tablet"><span class="ui_icon pencil-paper"></span><span class="badgetext">20</span><span class="ui_icon thumbs-up-fill"></span><span class="badgetext">11</span></div></div></div></div></div><div class="ui_column is-9"><span class="ui_bubble_rating bubble_50"></span><span class="ratingDate" title="5 November 2019">Reviewed yesterday </span><div class="quote isNew"><a href="/ShowUserReviews-g480193-d10147730-r724131885-Church_of_St_George-Lalibela_Amhara_Region.html" class="title " onclick="(ta.prwidgets.getjs(this,'handlers')).reviewClick(this.href, '0');" id="rn724131885"><span class="noQuotes">This site rival Petra and Machu Pichu come now and beat the crowds</span></a></div><div class="prw_rup prw_reviews_text_summary_hsx" data-prwidget-name="reviews_text_summary_hsx" data-prwidget-init="handlers"><div class="entry"><p class="partial_entry">This site is amazing. Dark passages and bridges link the buildings it feels like Indiana Jones might leap out at any time. Archaelogical excavation was taking place while we were there.</p></div></div><div class="prw_rup prw_reviews_stay_date_hsx" data-prwidget-name="reviews_stay_date_hsx" data-prwidget-init=""><span class="stay_date_label">Date of experience:</span> October 2019</div><div class="prw_rup prw_reviews_vote_line_hsx" data-prwidget-name="reviews_vote_line_hsx" data-prwidget-init="handlers" data-prwidget-deferred="deferred/clicks"><div class="tooltips wrap"><span id="ReportIAP_724131885" class="taLnk no_cpu ui_icon flag-fill" onclick="ta.prwidgets.call('deferred/clicks.clickReport',event,this,724131885,'member');return false;" onmouseover="widgetEvCall('handlers.hoverReport',event,this);return false;" data-tooltip="" data-position="above" data-content="Problem with this review?"></span></div><div class="helpful redesigned hsx_helpful"><span class="thankButton hsx_thank_button" onclick="ta.prwidgets.call('deferred/clicks.clickThank',event,this,'LeJIVqd4EVIpECri1GII2t6mbqgqguuuxizSxiniaxi2GqgbIIEJnISiCQQoqnQQeVsSVuqHydvKUAmqHMUKUdkkmKqHxfqHfGpEnGQQoqnQQiTQQoqnQQhIQQoqnQQzzViEyVqHRJcSHVcJQQoqnQQbaGJEJQQoqnQQeVySiCxiGIac6XoXmqoTpckXdUmUvvo0tEn1kXdUmUvvo0zH1dvKUAm0pSM17qnbxMUvbf7dqnqnfdd7mKoomX33XmdoUKvo0npEEeJIV1K0EJIVqiJcpV1U0Ii9VC1rZlU3XozxbZanTJUMxXrMZGZluZgsLHxbxECkxdiyLxMx2Xgyy3CzzCxGmuzCtvqI7zzfrbGqIKx2NGIx2yidxXl7nr9RUSqQTLaby7qQ2wv2f9CndRSKhqnqiHlx5kyR9qiviq8TTx5xbxXzsV5naxXSElqIUJsiGoZexbxG5GxEbq8sLDfGJObxGZTmxGdgzzxXxGm37zseHHzsxEJvmdRMlxEqIE');return false;"><span class="helpful_text"><span class="ui_icon thumbs-up-fill emphasizeWithColor"></span><span class="numHelp emphasizeWithColor"></span><span class="thankUser">Thank Harry H </span></span><div class="buttonShade hidden"><img src="https://static.tacdn.com/img2/generic/site/loading_anim_gry_sml.gif" alt=""></div></span></div></div><div class="loadingShade hidden"><div class="ui_spinner"></div></div></div></div></div></div>

In [7]:
!cat tripadvisor.json